In [ ]:
# Setup
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import warnings
import sys
warnings.filterwarnings('ignore')

# Detect environment (check if running in Google Colab)
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

# Mount Google Drive if in Colab
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    print(" Google Drive mounted")

# Optimization
try:
    import pygad
    PYGAD_AVAILABLE = True
except ImportError:
    print(" PyGAD not installed. Install with: pip install pygad")
    PYGAD_AVAILABLE = False

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

print(" Libraries loaded")
print(f" {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print(f" Environment: {'Google Colab' if IN_COLAB else 'Local'}")
print(f"PyGAD available: {PYGAD_AVAILABLE}")

✅ Libraries loaded
📅 2025-10-22 10:38
🖥️ Environment: Local
PyGAD available: True


In [ ]:
# Paths (works both locally and in Colab)
if IN_COLAB:
    BASE_PATH = Path('/content/drive/MyDrive/HACKATHON_DATALAB')
else:
    BASE_PATH = Path.cwd()

RESULTS_PATH = BASE_PATH / 'data' / 'results'
OUTPUT_PATH = BASE_PATH / 'optimization_outputs'
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

print(f" Results: {RESULTS_PATH}")
print(f" Output: {OUTPUT_PATH}")

📂 Results: /Users/fadybekkar/Desktop/EPITECH/HACK/Hackaton_Data/projet/data/results
📂 Output: /Users/fadybekkar/Desktop/EPITECH/HACK/Hackaton_Data/projet/optimization_outputs


In [ ]:
# Load predictions from forecasting notebook with error handling
predictions_file = RESULTS_PATH / 'demand_predictions.csv'

if predictions_file.exists():
    try:
        df_predictions = pd.read_csv(predictions_file)
        df_predictions['date'] = pd.to_datetime(df_predictions['date'])

        print(f" Loaded predictions: {df_predictions.shape}")
        print(f" Date range: {df_predictions['date'].min()} to {df_predictions['date'].max()}")
        print(f" Regions: {df_predictions['region'].nunique()}")

        # Validate data
        required_columns = ['date', 'region', 'predicted_demand']
        missing_columns = [col for col in required_columns if col not in df_predictions.columns]

        if missing_columns:
            print(f" ERROR: Missing required columns: {missing_columns}")
            df_predictions = None
        elif len(df_predictions) == 0:
            print(" ERROR: Predictions file is empty!")
            df_predictions = None
        else:
            print(" Data validation passed")
            print(f"\n Sample:")
            print(df_predictions.head())
    except Exception as e:
        print(f" ERROR loading predictions: {e}")
        print("Please check the file and try regenerating it in 03_Forecasting.ipynb")
        df_predictions = None
else:
    print(" Predictions file not found. Please run 03_Forecasting.ipynb first.")
    print(f"Expected location: {predictions_file}")
    print("\nℹ️ The forecasting notebook creates this file as its output.")
    df_predictions = None

✅ Loaded predictions: (869, 6)
📅 Date range: 2024-07-29 00:00:00 to 2025-10-06 00:00:00
🗺️ Regions: 17
✅ Data validation passed

👀 Sample:
        date                   region  \
0 2024-07-29  Auvergne et Rhône-Alpes   
1 2024-08-05  Auvergne et Rhône-Alpes   
2 2024-08-12  Auvergne et Rhône-Alpes   
3 2024-08-19  Auvergne et Rhône-Alpes   
4 2024-08-26  Auvergne et Rhône-Alpes   

   Taux de passages aux urgences pour grippe  predicted_demand  lower_bound  \
0                                  46.850947         66.676860          0.0   
1                                  54.274084         59.030876          0.0   
2                                  68.867361         66.142110          0.0   
3                                  56.764428         69.249565          0.0   
4                                  39.864928         65.843025          0.0   

   upper_bound  
0    385.29930  
1    377.65332  
2    384.76456  
3    387.87200  
4    384.46545  


In [ ]:
if df_predictions is not None:
    # Aggregate by region
    regional_demand = df_predictions.groupby('region').agg({
        'predicted_demand': 'sum',
        'lower_bound': 'sum',
        'upper_bound': 'sum'
    }).reset_index()

    regional_demand.columns = ['region', 'total_demand', 'demand_low', 'demand_high']
    regional_demand = regional_demand.sort_values('total_demand', ascending=False)

    print(" Regional Demand Summary:\n")
    print(regional_demand.to_string(index=False))

    total_demand = regional_demand['total_demand'].sum()
    print(f"\n Total predicted demand: {total_demand:,.0f} interventions")

    # Visualize
    fig = px.bar(
        regional_demand,
        x='region',
        y='total_demand',
        title=' Total Predicted Demand by Region',
        labels={'region': 'Region', 'total_demand': 'Total Demand'},
        color='total_demand',
        color_continuous_scale='Reds'
    )

    fig.update_layout(
        xaxis_tickangle=-45,
        height=500,
        template='plotly_white'
    )

    viz_path = BASE_PATH / 'visualizations'
    fig.write_html(viz_path / 'regional_demand.html')
    fig.show()
    print(f"\n Saved: regional_demand.html")

🗺️ Regional Demand Summary:

                    region  total_demand   demand_low  demand_high
                   Réunion  26084.277610 11336.337588  42334.02157
                    Guyane  17472.766140  7595.470669  27350.06178
             Île-de-France  15747.553640  3784.627004  31678.67551
                   Mayotte  15614.735885  3979.929923  34732.08155
                Martinique  15386.258741  5031.706105  32591.87027
                Guadeloupe  13223.890379  2739.741808  28517.76738
Provence-Alpes-Côte d'Azur  12672.381991  3598.828192  29559.37136
       Centre-Val de Loire  12492.780489  3075.809875  29379.76958
Bourgogne et Franche-Comté  12137.287606  2683.845632  28705.65406
                 Normandie  11299.198637  3146.069874  27867.56539
                 Grand Est  11088.278591  2772.655084  27338.02297
           Hauts-de-France  10709.609433  3190.509646  26959.35372
        Nouvelle Aquitaine  10419.617648  2436.116740  26987.98471
                  Bretagne   9869


✅ Saved: regional_demand.html


In [ ]:
if df_predictions is not None:
    # Define supply scenarios
    scenarios = {
        'Shortage (70%)': total_demand * 0.70,
        'Limited (85%)': total_demand * 0.85,
        'Adequate (100%)': total_demand * 1.00,
        'Surplus (120%)': total_demand * 1.20
    }

    print("💉 Vaccine Supply Scenarios:\n")
    for name, supply in scenarios.items():
        shortfall = total_demand - supply
        pct = (supply / total_demand) * 100
        status = '⚠️' if shortfall > 0 else '✅'
        print(f"{status} {name}: {supply:,.0f} vaccines ({pct:.0f}% of demand)")

    # We'll focus on the realistic "Limited" scenario
    TOTAL_SUPPLY = scenarios['Limited (85%)']
    print(f"\n Focus scenario: {TOTAL_SUPPLY:,.0f} vaccines (85% of demand)")

💉 Vaccine Supply Scenarios:

⚠️ Shortage (70%): 155,616 vaccines (70% of demand)
⚠️ Limited (85%): 188,963 vaccines (85% of demand)
✅ Adequate (100%): 222,309 vaccines (100% of demand)
✅ Surplus (120%): 266,771 vaccines (120% of demand)

🎯 Focus scenario: 188,963 vaccines (85% of demand)


In [ ]:
if df_predictions is not None:
    print("📐 Testing Simple Allocation Strategies...\n")

    allocation_results = {}

    # 1. Proportional Allocation (based on demand)
    regional_demand['allocation_proportional'] = (
        regional_demand['total_demand'] / total_demand * TOTAL_SUPPLY
    )

    regional_demand['unmet_proportional'] = (
        regional_demand['total_demand'] - regional_demand['allocation_proportional']
    ).clip(lower=0)

    regional_demand['coverage_proportional'] = (
        regional_demand['allocation_proportional'] / regional_demand['total_demand'] * 100
    )

    total_unmet_prop = regional_demand['unmet_proportional'].sum()
    avg_coverage_prop = regional_demand['coverage_proportional'].mean()

    allocation_results['Proportional'] = {
        'total_unmet': total_unmet_prop,
        'avg_coverage': avg_coverage_prop,
        'std_coverage': regional_demand['coverage_proportional'].std(),
        'min_coverage': regional_demand['coverage_proportional'].min()
    }

    print(f" Proportional Allocation:")
    print(f"   Total unmet demand: {total_unmet_prop:,.0f}")
    print(f"   Avg coverage: {avg_coverage_prop:.1f}%")
    print(f"   Coverage std: {regional_demand['coverage_proportional'].std():.1f}%\n")

    # 2. Equal Allocation (same amount per region)
    equal_allocation = TOTAL_SUPPLY / len(regional_demand)
    regional_demand['allocation_equal'] = equal_allocation

    regional_demand['unmet_equal'] = (
        regional_demand['total_demand'] - regional_demand['allocation_equal']
    ).clip(lower=0)

    regional_demand['coverage_equal'] = (
        regional_demand['allocation_equal'] / regional_demand['total_demand'] * 100
    )

    total_unmet_eq = regional_demand['unmet_equal'].sum()
    avg_coverage_eq = regional_demand['coverage_equal'].mean()

    allocation_results['Equal'] = {
        'total_unmet': total_unmet_eq,
        'avg_coverage': avg_coverage_eq,
        'std_coverage': regional_demand['coverage_equal'].std(),
        'min_coverage': regional_demand['coverage_equal'].min()
    }

    print(f" Equal Allocation:")
    print(f"   Total unmet demand: {total_unmet_eq:,.0f}")
    print(f"   Avg coverage: {avg_coverage_eq:.1f}%")
    print(f"   Coverage std: {regional_demand['coverage_equal'].std():.1f}%\n")

📐 Testing Simple Allocation Strategies...

✅ Proportional Allocation:
   Total unmet demand: 33,346
   Avg coverage: 85.0%
   Coverage std: 0.0%

✅ Equal Allocation:
   Total unmet demand: 40,977
   Avg coverage: 91.3%
   Coverage std: 22.2%



In [ ]:
if PYGAD_AVAILABLE and df_predictions is not None:
    print(" Running Genetic Algorithm Optimization...\n")

    # Extract demand as numpy array (for speed)
    demand_array = regional_demand['total_demand'].values
    n_regions = len(demand_array)

    # Define fitness function with better objective formulation
    def fitness_function(ga_instance, solution, solution_idx):
        """
        Multi-objective fitness function for vaccine allocation optimization.

        Objectives:
        1. Minimize unmet demand (efficiency) - we want to serve as much demand as possible
        2. Minimize inequality (equity) - fair distribution across regions
        3. Maximize minimum coverage (access) - no region left too far behind
        4. Penalize violations of physical constraints

        Returns:
            float: Fitness score (higher is better)
        """
        # Normalize solution to match total supply exactly
        # Prevent negative allocations
        solution = np.maximum(solution, 0)

        if solution.sum() == 0:
            return 0  # Invalid solution

        allocation = solution / solution.sum() * TOTAL_SUPPLY

        # Calculate coverage rates
        coverage = np.minimum(allocation / demand_array * 100, 100)  # Cap at 100%

        # Calculate unmet demand
        unmet = np.maximum(demand_array - allocation, 0)

        # Objective 1: Efficiency (minimize unmet demand)
        # Normalize to 0-1 range (1 = perfect, 0 = worst)
        efficiency = 1 - (unmet.sum() / demand_array.sum())

        # Objective 2: Equity (minimize standard deviation of coverage)
        # Penalize high inequality
        coverage_std = coverage.std()
        equity = 1 - (coverage_std / 100)  # Normalize
        equity = max(0, equity)  # Ensure non-negative

        # Objective 3: Access (maximize minimum coverage)
        # Ensure no region is left too far behind
        min_coverage = coverage.min()
        access = min_coverage / 100  # Normalize to 0-1

        # Objective 4: Bonus for regions above target threshold (80%)
        regions_above_target = (coverage >= 80).sum() / n_regions

        # Penalty for extreme allocations (avoid putting all eggs in one basket)
        max_allocation_share = allocation.max() / TOTAL_SUPPLY
        concentration_penalty = 0
        if max_allocation_share > 0.3:  # No region should get more than 30%
            concentration_penalty = (max_allocation_share - 0.3) * 0.5

        # Weighted combination of objectives
        # Adjust weights based on policy priorities
        fitness = (
            0.40 * efficiency +           # Primary: serve demand efficiently
            0.25 * equity +                # Important: fair distribution
            0.20 * access +                # Important: help struggling regions
            0.15 * regions_above_target -  # Bonus: many regions well-served
            concentration_penalty           # Penalty: avoid concentration
        )

        return fitness

    # Setup genetic algorithm
    ga_instance = pygad.GA(
        num_generations=200,
        num_parents_mating=10,
        fitness_func=fitness_function,
        sol_per_pop=50,
        num_genes=n_regions,
        init_range_low=0,
        init_range_high=1,
        parent_selection_type='tournament',
        keep_parents=2,
        crossover_type='uniform',
        mutation_type='random',
        mutation_percent_genes=10,
        random_seed=42
    )

    # Run optimization
    print(" Optimizing... (this may take a minute)")
    ga_instance.run()
    print(" Optimization complete!\n")

    # Get best solution
    solution, solution_fitness, solution_idx = ga_instance.best_solution()

    # Normalize to actual supply
    optimal_allocation = solution / solution.sum() * TOTAL_SUPPLY

    regional_demand['allocation_optimal'] = optimal_allocation
    regional_demand['unmet_optimal'] = (
        regional_demand['total_demand'] - regional_demand['allocation_optimal']
    ).clip(lower=0)
    regional_demand['coverage_optimal'] = (
        regional_demand['allocation_optimal'] / regional_demand['total_demand'] * 100
    )

    total_unmet_opt = regional_demand['unmet_optimal'].sum()
    avg_coverage_opt = regional_demand['coverage_optimal'].mean()

    allocation_results['Optimized (GA)'] = {
        'total_unmet': total_unmet_opt,
        'avg_coverage': avg_coverage_opt,
        'std_coverage': regional_demand['coverage_optimal'].std(),
        'min_coverage': regional_demand['coverage_optimal'].min()
    }

    print(f" Optimized Allocation:")
    print(f"   Total unmet demand: {total_unmet_opt:,.0f}")
    print(f"   Avg coverage: {avg_coverage_opt:.1f}%")
    print(f"   Coverage std: {regional_demand['coverage_optimal'].std():.1f}%")
    print(f"   Fitness score: {solution_fitness:.4f}")

else:
    print(" Skipping GA optimization (PyGAD not installed)")

🧬 Running Genetic Algorithm Optimization...

🚀 Optimizing... (this may take a minute)
✅ Optimization complete!

🏆 Optimized Allocation:
   Total unmet demand: 33,346
   Avg coverage: 85.2%
   Coverage std: 5.3%
   Fitness score: 0.8648


In [ ]:
if allocation_results:
    print("\n" + "="*80)
    print(" ALLOCATION STRATEGY COMPARISON")
    print("="*80)

    comparison = pd.DataFrame(allocation_results).T
    comparison = comparison.sort_values('total_unmet')

    print("\n" + comparison.to_string())

    # Identify best
    best_strategy = comparison['total_unmet'].idxmin()
    print(f"\n BEST STRATEGY: {best_strategy}")
    print(f"   Lowest unmet demand: {comparison.loc[best_strategy, 'total_unmet']:,.0f}")
    print(f"   Highest avg coverage: {comparison.loc[best_strategy, 'avg_coverage']:.1f}%")

    # Add interpretation for decision-makers
    print(f"\n What These Numbers Mean:")
    print(f"   • total_unmet: How many people DON'T get vaccinated (lower is better)")
    print(f"   • avg_coverage: Average % of demand met across regions (higher is better)")
    print(f"   • std_coverage: How much coverage varies between regions (lower = more equal)")
    print(f"   • min_coverage: The worst-served region's coverage (higher is better)")

    print(f"\n Key Insight:")
    if 'Optimized (GA)' in comparison.index:
        prop_unmet = comparison.loc['Proportional', 'total_unmet']
        opt_unmet = comparison.loc['Optimized (GA)', 'total_unmet']
        improvement = ((prop_unmet - opt_unmet) / prop_unmet) * 100
        print(f"   Optimized allocation serves {improvement:.0f}% more people than proportional!")
        print(f"   That's {prop_unmet - opt_unmet:,.0f} additional people vaccinated.")
        print(f"   → This translates to ~{(prop_unmet - opt_unmet) * 0.7:,.0f} fewer emergency visits!")

    # Save comparison
    comparison.to_csv(OUTPUT_PATH / 'allocation_comparison.csv')
    print(f"\n Comparison saved: allocation_comparison.csv")


📊 ALLOCATION STRATEGY COMPARISON

                 total_unmet  avg_coverage  std_coverage  min_coverage
Proportional    33346.364151     85.000000  8.702336e-15     85.000000
Optimized (GA)  33346.364151     85.175503  5.297007e+00     77.658673
Equal           40976.583950     91.277302  2.217784e+01     42.613619

🏆 BEST STRATEGY: Proportional
   Lowest unmet demand: 33,346
   Highest avg coverage: 85.0%

📊 What These Numbers Mean:
   • total_unmet: How many people DON'T get vaccinated (lower is better)
   • avg_coverage: Average % of demand met across regions (higher is better)
   • std_coverage: How much coverage varies between regions (lower = more equal)
   • min_coverage: The worst-served region's coverage (higher is better)

💡 Key Insight:
   Optimized allocation serves -0% more people than proportional!
   That's -0 additional people vaccinated.
   → This translates to ~-0 fewer emergency visits!

💾 Comparison saved: allocation_comparison.csv


In [ ]:
if df_predictions is not None:
    # Create final allocation plan
    columns_to_include = [
        'region', 'total_demand',
        'allocation_proportional', 'coverage_proportional',
    ]

    # Add optimal allocation columns if they exist (only if GA ran)
    if 'allocation_optimal' in regional_demand.columns:
        columns_to_include.extend(['allocation_optimal', 'coverage_optimal'])

    allocation_plan = regional_demand[columns_to_include].copy()

    # Round allocations
    allocation_plan['allocation_proportional'] = allocation_plan['allocation_proportional'].round(0)
    if 'allocation_optimal' in allocation_plan.columns:
        allocation_plan['allocation_optimal'] = allocation_plan['allocation_optimal'].round(0)

    # Sort by demand
    allocation_plan = allocation_plan.sort_values('total_demand', ascending=False)

    print("\n FINAL ALLOCATION PLAN:\n")
    print(allocation_plan.to_string(index=False))

    # Save
    allocation_plan.to_csv(OUTPUT_PATH / 'final_allocation_plan.csv', index=False)
    print(f"\n Saved: final_allocation_plan.csv")


📋 FINAL ALLOCATION PLAN:

                    region  total_demand  allocation_proportional  coverage_proportional  allocation_optimal  coverage_optimal
                   Réunion  26084.277610                  22172.0                   85.0             22121.0         84.804325
                    Guyane  17472.766140                  14852.0                   85.0             14102.0         80.709509
             Île-de-France  15747.553640                  13385.0                   85.0             13597.0         86.345451
                   Mayotte  15614.735885                  13273.0                   85.0             12417.0         79.518079
                Martinique  15386.258741                  13078.0                   85.0             12863.0         83.603673
                Guadeloupe  13223.890379                  11240.0                   85.0             13075.0         98.873597
Provence-Alpes-Côte d'Azur  12672.381991                  10772.0                   

In [ ]:
if df_predictions is not None:
    # Coverage comparison
    fig = go.Figure()

    # Demand baseline
    fig.add_trace(go.Bar(
        x=regional_demand['region'],
        y=regional_demand['total_demand'],
        name='Total Demand',
        marker=dict(color='lightgray')
    ))

    # Proportional allocation
    fig.add_trace(go.Bar(
        x=regional_demand['region'],
        y=regional_demand['allocation_proportional'],
        name='Proportional Allocation',
        marker=dict(color='blue', opacity=0.6)
    ))

    # Optimal allocation (if available)
    if 'allocation_optimal' in regional_demand.columns:
        fig.add_trace(go.Bar(
            x=regional_demand['region'],
            y=regional_demand['allocation_optimal'],
            name='Optimized Allocation',
            marker=dict(color='green', opacity=0.6)
        ))

    fig.update_layout(
        title=' Vaccine Allocation by Region',
        xaxis_title='Region',
        yaxis_title='Vaccines',
        barmode='group',
        xaxis_tickangle=-45,
        height=500,
        template='plotly_white'
    )

    fig.write_html(viz_path / 'allocation_by_region.html')
    fig.show()
    print("\n Saved: allocation_by_region.html")


✅ Saved: allocation_by_region.html


In [ ]:
# Coverage comparison
if df_predictions is not None:
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=regional_demand['region'],
        y=regional_demand['coverage_proportional'],
        mode='markers+lines',
        name='Proportional',
        marker=dict(size=10, color='blue')
    ))

    if 'coverage_optimal' in regional_demand.columns:
        fig.add_trace(go.Scatter(
            x=regional_demand['region'],
            y=regional_demand['coverage_optimal'],
            mode='markers+lines',
            name='Optimized',
            marker=dict(size=10, color='green')
        ))

    # Target line at 80%
    fig.add_hline(
        y=80,
        line_dash='dash',
        line_color='red',
        annotation_text='Target: 80% Coverage',
        annotation_position='right'
    )

    fig.update_layout(
        title=' Coverage Rate by Region',
        xaxis_title='Region',
        yaxis_title='Coverage (%)',
        xaxis_tickangle=-45,
        height=500,
        template='plotly_white',
        hovermode='x unified'
    )

    fig.write_html(viz_path / 'coverage_by_region.html')
    fig.show()
    print(" Saved: coverage_by_region.html")

✅ Saved: coverage_by_region.html


In [ ]:
if df_predictions is not None:
    print("\n COST-BENEFIT ANALYSIS\n" + "="*80)

    # Assumptions (adjust based on real data)
    COST_PER_VACCINE = 15  # euros
    COST_PER_EMERGENCY_VISIT = 200  # euros
    EFFECTIVENESS = 0.7  # vaccines 70% effective at preventing emergency visits

    # Calculate for proportional allocation
    vaccines_given_prop = regional_demand['allocation_proportional'].sum()
    visits_prevented_prop = vaccines_given_prop * EFFECTIVENESS

    cost_prop = vaccines_given_prop * COST_PER_VACCINE
    savings_prop = visits_prevented_prop * COST_PER_EMERGENCY_VISIT
    net_benefit_prop = savings_prop - cost_prop
    roi_prop = (net_benefit_prop / cost_prop) * 100

    print(" Proportional Allocation:")
    print(f"   Vaccines given: {vaccines_given_prop:,.0f}")
    print(f"   Emergency visits prevented: {visits_prevented_prop:,.0f}")
    print(f"   Cost: €{cost_prop:,.0f}")
    print(f"   Savings: €{savings_prop:,.0f}")
    print(f"   Net benefit: €{net_benefit_prop:,.0f}")
    print(f"   ROI: {roi_prop:.1f}%\n")

    # Calculate for optimal allocation (if available)
    if 'allocation_optimal' in regional_demand.columns:
        vaccines_given_opt = regional_demand['allocation_optimal'].sum()
        visits_prevented_opt = vaccines_given_opt * EFFECTIVENESS

        cost_opt = vaccines_given_opt * COST_PER_VACCINE
        savings_opt = visits_prevented_opt * COST_PER_EMERGENCY_VISIT
        net_benefit_opt = savings_opt - cost_opt
        roi_opt = (net_benefit_opt / cost_opt) * 100

        print(" Optimized Allocation:")
        print(f"   Vaccines given: {vaccines_given_opt:,.0f}")
        print(f"   Emergency visits prevented: {visits_prevented_opt:,.0f}")
        print(f"   Cost: €{cost_opt:,.0f}")
        print(f"   Savings: €{savings_opt:,.0f}")
        print(f"   Net benefit: €{net_benefit_opt:,.0f}")
        print(f"   ROI: {roi_opt:.1f}%\n")

        improvement = net_benefit_opt - net_benefit_prop
        print(f" Optimization improvement: €{improvement:,.0f} additional net benefit")

    print("\n" + "="*80)
    print("\n Every €1 spent on vaccines saves €{:.2f} in emergency costs!".format(roi_prop/100 + 1))


💰 COST-BENEFIT ANALYSIS
📊 Proportional Allocation:
   Vaccines given: 188,963
   Emergency visits prevented: 132,274
   Cost: €2,834,441
   Savings: €26,454,782
   Net benefit: €23,620,341
   ROI: 833.3%

🏆 Optimized Allocation:
   Vaccines given: 188,963
   Emergency visits prevented: 132,274
   Cost: €2,834,441
   Savings: €26,454,782
   Net benefit: €23,620,341
   ROI: 833.3%

💡 Optimization improvement: €0 additional net benefit


✅ Every €1 spent on vaccines saves €9.33 in emergency costs!


In [ ]:
if df_predictions is not None:
    # Create executive summary
    summary_lines = []
    summary_lines.append("#  Vaccine Distribution Optimization - Executive Summary\n\n")
    summary_lines.append(f"**Date**: {datetime.now().strftime('%Y-%m-%d %H:%M')}\n\n")
    summary_lines.append("---\n\n")

    summary_lines.append("##  Demand Forecast\n\n")
    summary_lines.append(f"- **Total predicted demand**: {total_demand:,.0f} interventions\n")
    summary_lines.append(f"- **Regions analyzed**: {len(regional_demand)}\n")
    summary_lines.append(f"- **Time horizon**: {df_predictions['date'].nunique()} weeks\n\n")

    summary_lines.append("##  Supply Scenario\n\n")
    summary_lines.append(f"- **Available supply**: {TOTAL_SUPPLY:,.0f} vaccines\n")
    summary_lines.append(f"- **Coverage level**: 85% of predicted demand\n")
    summary_lines.append(f"- **Shortfall**: {total_demand - TOTAL_SUPPLY:,.0f} vaccines\n\n")

    summary_lines.append("##  Recommended Strategy\n\n")
    if 'allocation_optimal' in regional_demand.columns:
        summary_lines.append("**Optimized allocation** (genetic algorithm)\n\n")
        summary_lines.append(f"- Total unmet demand: {total_unmet_opt:,.0f}\n")
        summary_lines.append(f"- Average coverage: {avg_coverage_opt:.1f}%\n")
        summary_lines.append(f"- Regions above 80%: {(regional_demand['coverage_optimal'] >= 80).sum()} of {len(regional_demand)}\n\n")
    else:
        summary_lines.append("**Proportional allocation** (baseline)\n\n")
        summary_lines.append(f"- Total unmet demand: {total_unmet_prop:,.0f}\n")
        summary_lines.append(f"- Average coverage: {avg_coverage_prop:.1f}%\n\n")

    summary_lines.append("##  Economic Impact\n\n")
    summary_lines.append(f"- **Investment**: €{cost_prop:,.0f}\n")
    summary_lines.append(f"- **Expected savings**: €{savings_prop:,.0f}\n")
    summary_lines.append(f"- **Net benefit**: €{net_benefit_prop:,.0f}\n")
    summary_lines.append(f"- **ROI**: {roi_prop:.0f}%\n\n")

    summary_lines.append("##  Priority Regions\n\n")
    top_5 = regional_demand.nlargest(5, 'total_demand')
    summary_lines.append("**Highest demand (priority for allocation)**:\n\n")
    for idx, row in top_5.iterrows():
        summary_lines.append(f"- {row['region']}: {row['total_demand']:,.0f} predicted interventions\n")

    summary_lines.append("\n##  Implementation Steps\n\n")
    summary_lines.append("1. **Secure supply**: Order {TOTAL_SUPPLY:,.0f} vaccine doses\n".format(TOTAL_SUPPLY=TOTAL_SUPPLY))
    summary_lines.append("2. **Distribute**: Allocate according to optimal plan (see `final_allocation_plan.csv`)\n")
    summary_lines.append("3. **Monitor**: Track actual demand vs predictions weekly\n")
    summary_lines.append("4. **Adjust**: Reallocate surplus from low-demand regions to high-demand regions\n")
    summary_lines.append("5. **Evaluate**: Measure effectiveness after campaign completion\n\n")

    summary_lines.append("---\n\n")
    summary_lines.append("##  Outputs Generated\n\n")
    summary_lines.append("- `final_allocation_plan.csv` - Detailed regional allocations\n")
    summary_lines.append("- `allocation_comparison.csv` - Strategy comparison metrics\n")
    summary_lines.append("- `allocation_by_region.html` - Interactive visualization\n")
    summary_lines.append("- `coverage_by_region.html` - Coverage rate visualization\n\n")

    # Save
    summary_path = OUTPUT_PATH / 'executive_summary.md'
    with open(summary_path, 'w', encoding='utf-8') as f:
        f.writelines(summary_lines)

    print(f"\n Executive summary saved: {summary_path}")


✅ Executive summary saved: /Users/fadybekkar/Desktop/EPITECH/HACK/Hackaton_Data/projet/optimization_outputs/executive_summary.md
